In [95]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [96]:
import numpy as np
# axes = (up/down, left/right, back/front, faces)
# faces = the colors of (up/down, left/right, back/front)

top    = 0, slice(0,3), slice(0,3), 0 # == cube[0,:,:,0]
bottom = 2, slice(0,3), slice(0,3), 0 
left   = slice(0,3), 0, slice(0,3), 1
right  = slice(0,3), 2, slice(0,3), 1
front  = slice(0,3), slice(0,3), 0, 2
back   = slice(0,3), slice(0,3), 2, 2

a = np.zeros(3*3*3*3, dtype='str').reshape([3,3,3,3])
a[:,:,:,:] = '-'
a[top] = 'W'
a[bottom] = 'Y'
a[left] = 'O'  
a[right] = 'R' 
a[front] = 'G' 
a[back] = 'B'  

# test cube
letters = [chr(i) for i in range(ord('a'), ord('z') + 1)] + [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('0'), ord('9') + 1)]
for i in range(len([top, bottom, left, front, right, back])):
    face = [top, bottom, left, front, right, back][i]
    a[face] = np.array(letters[9*i:9*(i+1)]).reshape([3,3])

In [97]:
def p(a=a): # Print the cube
    blank = np.zeros(3*3, dtype='str').reshape([3,3]); blank[:,:] = ' '
    strip = np.zeros(3*1, dtype='str').reshape([3,1]); strip[:,:] = ' '
    c = np.concatenate((
        np.concatenate((blank, strip, np.flip(np.transpose(a[top]), axis=0), strip, blank, strip, blank), axis=1),
        np.concatenate((np.flip(a[left], axis=1), strip, a[front], strip, a[right], strip, np.flip(a[back], axis=1)), axis=1),
        np.concatenate((blank, strip, np.transpose(a[bottom]), strip, blank, strip, blank), axis=1)
    ))
    for i in range(c.shape[0]):
        print(' '.join(c[i,:]) )
p()

        c f i                
        b e h                
        a d g                
u t s   B C D   K L M   V U T
x w v   E F G   N O P   Y X W
A z y   H I J   Q R S   1 0 Z
        j m p                
        k n q                
        l o r                


In [98]:
def p4(a=a): # Print the 4D hypercube as a 3x3 grid of 3x3 grids
    blank = np.zeros(3*3 + 2, dtype='str').reshape([1,11]); blank[:,:] = ' '
    strip = np.zeros(3*1, dtype='str').reshape([3,1]); strip[:,:] = ' '
    c = np.concatenate((
        np.concatenate([a[0,0,:,:], strip, a[0,1,:,:], strip, a[0,2,:,:]], axis=1), blank,
        np.concatenate([a[1,0,:,:], strip, a[1,1,:,:], strip, a[1,2,:,:]], axis=1), blank,
        np.concatenate([a[2,0,:,:], strip, a[2,1,:,:], strip, a[2,2,:,:]], axis=1)
    ))
    for i in range(c.shape[0]):
        print(' '.join(c[i]) )
    print('----')

p4()

a s B   d - C   g K D
b t -   e - -   h L -
c u T   f - U   i M V
                     
- v E   - - F   - N G
- w -   - - -   - O -
- x W   - - X   - P Y
                     
j y H   m - I   p Q J
k z -   n - -   q R -
l A Z   o - 0   r S 1
----


In [99]:
# Rotate the top clockwise
def rotate_top(cube):
    b=cube.copy()
    b[0,:,:,:]=np.rot90(b[0,:,:,:], axes=[0, 1], k=-1)
    #p4(b)
    c=b.copy()
    c[0,:,:,1] = b[0,:,:,2]
    c[0,:,:,2] = b[0,:,:,1]
    return c

p(a)
p(rotate_top(a))

        c f i                
        b e h                
        a d g                
u t s   B C D   K L M   V U T
x w v   E F G   N O P   Y X W
A z y   H I J   Q R S   1 0 Z
        j m p                
        k n q                
        l o r                
        a b c                
        d e f                
        g h i                
B C D   K L M   V U T   u t s
x w v   E F G   N O P   Y X W
A z y   H I J   Q R S   1 0 Z
        j m p                
        k n q                
        l o r                


In [116]:
p4(a)
def reorient(cube, newTop):
    ax=[0,1,2,3]
    roll = 0
    k=1
    match newTop:
        case 'U': ax=[0,1]; k=0; roll=0
        case 'D': ax=[1,0]; k=2; roll=0
        case 'L': ax=[1,0]; k=1; roll=-1
        case 'R': ax=[1,0]; k=3; roll=-1
        case 'F': ax=[2,0]; k=1; roll=-1
        case 'B': ax=[2,0]; k=3; roll=-1
    #print(cube[0,0,0,:], cube[2,2,2,:])
    cube = np.rot90(cube, k=k, axes=ax)
    #p(cube)
    old = cube.copy()
    cube = cube.copy()
    match newTop:  # Reorient the fourth/faces dimension
        case 'L' | 'R': 
            cube[:,:,:,0] = old[:,:,:,1]
            cube[:,:,:,1] = old[:,:,:,0]
        case 'F' | 'B': 
            cube[:,:,:,0] = old[:,:,:,2]
            cube[:,:,:,2] = old[:,:,:,0]
    #p(cube)
    #print(cube[2,2,2,:])
    return cube
# axes = (up/down, left/right, back/front, faces)
# faces = the colors of (up/down, left/right, back/front)
reorient(a, 'B')

a s B   d - C   g K D
b t -   e - -   h L -
c u T   f - U   i M V
                     
- v E   - - F   - N G
- w -   - - -   - O -
- x W   - - X   - P Y
                     
j y H   m - I   p Q J
k z -   n - -   q R -
l A Z   o - 0   r S 1
----


array([[[['T', 'u', 'c'],
         ['W', 'x', '-'],
         ['Z', 'A', 'l']],

        [['U', '-', 'f'],
         ['X', '-', '-'],
         ['0', '-', 'o']],

        [['V', 'M', 'i'],
         ['Y', 'P', '-'],
         ['1', 'S', 'r']]],


       [[['-', 't', 'b'],
         ['-', 'w', '-'],
         ['-', 'z', 'k']],

        [['-', '-', 'e'],
         ['-', '-', '-'],
         ['-', '-', 'n']],

        [['-', 'L', 'h'],
         ['-', 'O', '-'],
         ['-', 'R', 'q']]],


       [[['B', 's', 'a'],
         ['E', 'v', '-'],
         ['H', 'y', 'j']],

        [['C', '-', 'd'],
         ['F', '-', '-'],
         ['I', '-', 'm']],

        [['D', 'K', 'g'],
         ['G', 'N', '-'],
         ['J', 'Q', 'p']]]], dtype='<U1')

In [124]:
#udlrfb
#L = Clockwise, L' = counterclockwise
p(a)
def perm(face, cube):
    match face[0]:
        case 'U': x=('U', 'U')
        case 'D': x=('D', 'D')
        case 'L': x=('L', 'R')
        case 'R': x=('R', 'L')
        case 'F': x=('F', 'B')
        case 'B': x=('B', 'F')
    b=cube.copy()    
    b = reorient(b, x[0])
    #p(b)
    print(x[0])
    for i in range(1 if len(face) == 1 else 3):
        b = rotate_top(b)
        print("hi!")
    #p(b)
    b = reorient(b, x[1]) 
    return b

p(perm("B", a))

        c f i                
        b e h                
        a d g                
u t s   B C D   K L M   V U T
x w v   E F G   N O P   Y X W
A z y   H I J   Q R S   1 0 Z
        j m p                
        k n q                
        l o r                
B
hi!
        M P S                
        b e h                
        a d g                
i t s   B C D   K L r   1 Y V
f w v   E F G   N O o   0 X U
c z y   H I J   Q R l   Z W T
        j m p                
        k n q                
        u x A                
